# Download Data

In [15]:
import requests
import pandas as pd
from io import BytesIO

def download_csv(csv_url):
    response = requests.get(csv_url)
    response.raise_for_status()

    try:
        df = pd.read_csv(BytesIO(response.content), sep=',')
    except pd.errors.ParserError:
        df = pd.read_csv(BytesIO(response.content), sep=';', error_bad_lines=False)

    return df

In [16]:
recipes_df = download_csv("https://github.com/GiovTemp/SustainaMeal_Case_Study/raw/main/data/final_recipes_set.csv")

In [17]:
recipes_df

,title,sustainability_score,sustainability_label,Unnamed: 0,recipe_id,description,author_id,duration,directions,ingredients,...,protein [g],direction_size,ingredients_sizes,who_score,fsa_score,nutri_score,normalization_comment,ingredient_food_kg_urls,ingredient_food_kg_names,healthiness_label
0,Jack O Lantern 'o Fragrance,0.002409,0,495656,466123,I suppose that this isn't really a recipe as m...,169430,30.0,['When you cut the lid out of the pumpkin do ...,"['pumpkin liking', 'candle pumpkin', 'pumpkin ...",...,0.6,7,4,0.302381,0.250,0.25,NaN,NaN,NaN,0
1,Boiled Radishes,0.002502,0,294268,179840,These taste a lot like new potatoes. The boil...,254469,22.0,"['Trim ends and bad spots off Radishes.', 'Boi...","['radishes size bag bunch', 'water radishes']",...,0.8,2,2,0.293040,1.000,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['applesauce', 'garlic clove', 'ginger', 'mola...",0
2,Horseradish Applesauce,0.002590,0,364381,174389,"I'm not sure this even qualifies as a recipe, ...",226918,22.0,"['Blend applesauce and horseradish. ', 'Refrig...","['applesauce', 'horseradish']",...,0.3,2,2,0.313757,0.875,0.50,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['all - purpose flour', 'butter', 'butternut s...",0
3,Survival Necklace,0.002623,0,43445,95937,Fun for kids to make and a great way to keep t...,25792,10.0,['Other supplies: 18 inches string or yarn or ...,['Fruit Loops cereal cup apple jacks kind hole...,...,8.8,7,2,0.319215,0.750,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Old Bay Seasoning', 'Worcestershire sauce', ...",0
4,Apple Cider Reduction,0.002623,0,59722,521756,I saw a post about this on Facebook and gave i...,171084,155.0,"['In 6-8 quart stock pan, measure out 2 cups o...","['gallon apple cider fresh', 'gallon apple cid...",...,0.0,10,2,0.214286,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['bean sprouts', 'chicken thighs', 'fresh ging...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100865,Curried Lamb on Rice,0.951024,2,103145,373244,"With modification this recipe is from ""The Dia...",133174,65.0,"['Over medium heat, melt the butter in a large...","['lamb', 'medium onion', 'cloves', 'butter', '...",...,28.7,5,13,0.223549,0.625,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Dijon mustard', 'Polish sausage', 'apricot p...",1
100866,Rich Lamb Curry,0.960504,2,442694,268247,The recipe isn't as time consuming as it looks...,491437,110.0,"['Heat 1 tbsp olive oil in dutch oven. ', 'Co...","['oil', 'lamb', 'flour', 'salt pepper', 'onion...",...,36.1,9,21,0.153328,0.000,0.00,NaN,NaN,NaN,1
100867,"Middle Eastern Slow-Cooked Stew With Lamb, Chi...",0.968458,2,359960,144850,From Cooking Light. Per 3/4 c. serving: 310 ca...,37779,112.0,['Let the oil get heating in a large pot over ...,"['oil', 'lamb', 'onions rings', 'water', 'clov...",...,20.8,12,19,0.205566,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['anchovies', 'celery ribs', 'diced tomatoes',...",1
100868,Five Meat Chili Con Carne With Beans,0.971454,2,37637,13568,yum,20571,210.0,"['In a large pot brown meat in stages, about a...","['beef', 'pork', 'lamb', 'sausage', 'sausage',...",...,14.5,7,15,0.197568,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"[""French\\'s French fried onions"", 'crabmeat',...",1


# SustainaMeal

Initializes the system by loading the data and preparing the embeddings.

In [18]:
from HeASe.sustainameal import SustainaMeal

nutrients = ['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]']

sm = SustainaMeal(
    recipes_df=recipes_df,
    nutrients=['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]'],
    transformer_name='davanstrien/autotrain-recipes-2451975973'
)

Processing Titles embeddings: 100%|██████████| 100870/100870 [14:48<00:00, 113.54batch/s]


# Upload Recipes of Experiments

In [19]:
import pandas as pd

# Paths to the JSON files

sustainability_label_2_recipes_file = '../experiments/data/sustainability_label_2_recipes.json'

# Read the JSON files into pandas DataFrames

sustainability_label_2_recipes = pd.read_json(sustainability_label_2_recipes_file)


# Compare LLM Rerank

In [20]:
sm.setup_llama_model_id()
sm.create_agent()

Loading checkpoint shards: 100%|██████████| 4/4 [01:20<00:00, 20.23s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
acceptable_tags = ['appetizers', 'main-dish', 'side-dishes', 'drinks', 'beverages', 'fruits', 'desserts',
                   'breakfast', 'pasta-rice-and-grains', 'rice', 'pasta', 'pizza', 'breads', 'meat', 'fish',
                   'seafood', 'beef', 'chicken', 'vegetarian']

In [22]:
nutrients_string = ", ".join(nutrients)
print(nutrients_string)

calories [cal], totalFat [g], saturatedFat [g], cholesterol [mg], sodium [mg], dietaryFiber [g], sugars [g], protein [g]


In [23]:
selected_recipes = pd.concat([sustainability_label_2_recipes])

In [24]:
len(selected_recipes)

100

In [25]:
# Definisci le soglie
WHO_SCORE_THRESHOLDS = ( 0.15, 0.25)  # (soglia media, soglia alta)
SUSTAINABILITY_SCORE_THRESHOLDS = (0.1, 0.5)  # (soglia media, soglia alta)

# Funzioni di aiuto per calcolare le label
def calculate_healthiness_label(who_score):
    if who_score > WHO_SCORE_THRESHOLDS[1]:
        return "0"
    elif who_score < WHO_SCORE_THRESHOLDS[0]:
        return "2"
    else:
        return "1"

def calculate_sustainability_label(sustainability_score):
    if sustainability_score <= SUSTAINABILITY_SCORE_THRESHOLDS[1]:
        return "0"
    elif sustainability_score >= SUSTAINABILITY_SCORE_THRESHOLDS[1]:
        return "2"
    else:
        return "1"

# Calcola le label in base ai valori di who_score e sustainability_score

In [26]:
similar_recipes = sm.find_similar_recipes("Pasta", 10, acceptable_tags=acceptable_tags, match_all_tags=False)

Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 82.00batch/s]

Tags to match: ['main-dish', 'pasta', 'vegetarian', 'pasta-rice-and-grains']
Number of filtered recipes: 43799


In [27]:
order_recipes = sm.order_recipe_by_sustainameal()
print(order_recipes)

                                                   title  similarity_score  \
15954                             Brown Rice and Edamame          0.983306   
16511  Tofu With Red Curry Paste, Peas, and Yellow To...          0.986884   
11836                Pasta with Tomatoes, Peas and Cream          0.989764   
29501                                          Tofu Bowl          0.982852   
50905                       Asparagus-Mushroom Primavera          0.988871   
64425         Angel Hair With Tomatoes, Basil and Garlic          0.983267   
70774                  Spaghettini With Smothered Onions          0.983303   
65074                    Orlando Bloom's Pasta Au Pistou          0.985402   
73434               Mini Farfalle With Tomatoes and Corn          0.991559   
85469                Pasta and Potatoes (Pasta e Patate)          0.984727   

       who_score  healthiness_increment  sustainability_score  \
15954   0.345328              47.711098              0.909384   
16511   0.3

In [28]:
# non ne capisco il senso
order_recipes_llama = sm.order_recipe_by_sustainameal()
print(order_recipes_llama)

                                                   title  similarity_score  \
15954                             Brown Rice and Edamame          0.983306   
16511  Tofu With Red Curry Paste, Peas, and Yellow To...          0.986884   
11836                Pasta with Tomatoes, Peas and Cream          0.989764   
29501                                          Tofu Bowl          0.982852   
50905                       Asparagus-Mushroom Primavera          0.988871   
64425         Angel Hair With Tomatoes, Basil and Garlic          0.983267   
70774                  Spaghettini With Smothered Onions          0.983303   
65074                    Orlando Bloom's Pasta Au Pistou          0.985402   
73434               Mini Farfalle With Tomatoes and Corn          0.991559   
85469                Pasta and Potatoes (Pasta e Patate)          0.984727   

       who_score  healthiness_increment  sustainability_score  \
15954   0.345328              47.711098              0.909384   
16511   0.3

In [29]:
best_choice_by_llama = sm.choose_best_recipe_with_llama()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [30]:
print(best_choice_by_llama)

Let's go with the Brown Rice and Edamame recipe. This dish is a great choice for a healthy and sustainable meal because it's made with simple, whole foods that are easy on the planet. Brown rice is a great source of fiber and nutrients, and edamame is high in protein and low in calories. This recipe is also very versatile - you can customize it with your favorite seasonings and add other vegetables like carrots or bell peppers to make it even more nutritious. Plus, brown rice and edamame are both relatively low-cost ingredients, making this recipe a budget-friendly option. Overall, this dish is a great way to nourish your body and the environment with minimal fuss.
